In [27]:
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_bootstrap_components as dbc
import dash_html_components as html
from dash.dependencies import Input, Output, State
import plotly.express as px

import pandas as pd
import numpy as np
import pyreadstat
import re


In [2]:
# Load Data

fpath = 'data/ATP W42.sav'

df, meta = pyreadstat.read_sav(fpath)

In [3]:
# Used meta.column_names to find questionnaire items according to thematic subject matter code (e.g. starts with 'RQ')
# Created a quick little helper function to group these theme codes into lists

def list_helper(theme_code):
    return [i for i in df.columns if theme_code in i]

past_future = ['PAST_W42', 'FUTURE_W42']
society = ['SC1_W42']
policy = list_helper('POLICY')
confidence = list_helper('CONF')
rq_form1 = list_helper('RQ')
pw_form2 = list_helper('PQ')
scm = list_helper('SCM')
q = [i for i in df.columns if re.search("^Q[0-9]", i)]
pop = list_helper('POP')
knowledge = list_helper('KNOW')
demographics = list_helper('F_')
weight = ['WEIGHT_W42']

In [4]:
# Changes the values in the dataframe according to the value formats in the metadata.
# Because the data is Categorical or Ordinal, any functions we perform won't require the original float values

df_copy = pyreadstat.pyreadstat.set_value_labels(df, meta)

In [20]:
# Dictionary of column names (abbreviations) and corresponding column values (instruction on questionnaire)
# list comprehension is for dcc.Dropdown 'options' parameter
meta_list = zip(meta.column_names, meta.column_labels)
policy_dropdown = [{'label': y, 'value': x} for x, y in meta_list if x in policy]


# Note: had to redeclare meta_list for second list comprehension. Returns empty list otherwise

meta_list = zip(meta.column_names, meta.column_labels)
demo_dropdown = [{'label': y, 'value': x} for x, y in meta_list if x in demographics]

##### Example: 
meta_dict['SC1_W42']
##### Output:
'SC1. Overall, would you say science has had a mostly positive effect on our society or a mostly negative effect on our society?'

In [45]:
external_stylesheets = [dbc.themes.LUMEN]

app = JupyterDash(__name__, external_stylesheets = external_stylesheets)


app.layout = html.Div([
        dbc.Container([
        
        html.Div([
            dcc.Dropdown(
                id = 'xaxis-column',
                options = demo_dropdown,
                value = 'F_AGECAT'
            )
        ]),
        html.Div([
            dcc.Dropdown(
                id = 'yaxis-column',
                options = policy_dropdown,
                value = 'POLICY1_W42'
            )
        ]),
    
    dcc.Graph(id='indicator-bar')
    ]),

])


@app.callback(
    Output('indicator-bar', 'figure'),
    [Input('xaxis-column', 'value'),
    Input('yaxis-column', 'value')])
def update_graph(x_axis, y_axis):
    new_df = pd.crosstab(df_copy[x_axis],
                         df_copy[y_axis],
                         df_copy.WEIGHT_W42, aggfunc = sum, dropna=True,
                         normalize='index'). \
                         loc[meta.variable_value_labels[x_axis].values()]. \
                         loc[:, meta.variable_value_labels[y_axis].values()]*100
    
    fig = px.bar(new_df, x=new_df.columns, y=test.index )

fig.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.02,
    xanchor="right",
    x=1
))
    
    return fig


if __name__ == '__main__':
    app.run_server(debug=True, inline_exceptions=True)

Dash app running on http://127.0.0.1:8050/


In [15]:
# def crosstab_frequency2(array1, array2):
#     return pd.crosstab(df_copy[array1],
#                        df_copy[array2],
#                        df_copy.WEIGHT_W42, aggfunc = sum, dropna=True,
#                        normalize='index'). \
#                        loc[meta.variable_value_labels[array1].values()]. \
#                        loc[:, meta.variable_value_labels[array2].values()]*100

# table = crosstab_frequency2(scm[-1], demographics[7])

In [16]:
# fig = px.bar(table)

In [17]:
# fig